In [1]:
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.3 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/smarginatura/makemore.git

fatal: destination path 'makemore' already exists and is not an empty directory.


In [ ]:
%cd makemore
!ls -l

/content/makemore
total 52
drwxr-xr-x 2 root root 4096 Jun  4 21:05 data
-rw-r--r-- 1 root root 3481 Jun  4 21:05 data_loader.py
drwxr-xr-x 2 root root 4096 Jun  4 21:05 experiments
-rw-r--r-- 1 root root 1072 Jun  4 21:05 LICENSE
-rw-r--r-- 1 root root 5446 Jun  4 21:19 model.py
drwxr-xr-x 2 root root 4096 Jun  4 21:05 out
drwxr-xr-x 2 root root 4096 Jun  4 21:14 __pycache__
-rw-r--r-- 1 root root    5 Jun  4 21:05 README.md
-rw-r--r-- 1 root root 8723 Jun  4 21:05 train.py
drwxr-xr-x 8 root root 4096 Jun  4 23:49 wandb


In [ ]:
import os, sys, time, math, random

import numpy as np
import torch
from torch.nn import functional as F

from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

from dataclasses import dataclass

from tqdm.auto import tqdm

In [ ]:
from model import ModelConfig, RNN, loss_fn

from data_loader import create_datasets, InfiniteDataLoader

In [ ]:
@dataclass
class TrainArgs:

    # system I/O
    input_file: str = 'data/names.txt'
    work_dir: str = 'out'
    resume: bool = False
    sample_only: bool = False
    num_workers: int = 4
    max_steps: int = -1
    device: str = 'cpu'
    seed: int = 3407

    # wandb logging
    wandb_log: bool = False # disabled by default
    wandb_project: str = 'name-generation'
    wandb_run_name: str = None

    # sampling
    top_k: int = -1 # -1 means no top-k

    # model
    cell_type: str = 'rnn'
    n_layer: int = 4
    n_embd: int = 64
    n_embd2: int = 64

    # optimization hyperparameter
    batch_size: int = 32
    learning_rate: float = 5e-4
    weight_decay: float = 0.01

# Training

In [ ]:
# Utils
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
args = TrainArgs(cell_type='rnn',
                 device='cuda' if torch.cuda.is_available() else 'cpu',
                 max_steps=50_000,
                 batch_size = 32,
                 learning_rate=1e-3,
                 weight_decay=1e-2,
                 num_workers=4,
                 wandb_log=True)

set_seed(args.seed)
os.makedirs(args.work_dir, exist_ok=True)

In [ ]:
vars(args)

{'input_file': 'data/names.txt',
 'work_dir': 'out',
 'resume': False,
 'sample_only': False,
 'num_workers': 4,
 'max_steps': 50000,
 'device': 'cpu',
 'seed': 3407,
 'wandb_log': True,
 'wandb_project': 'name-generation',
 'wandb_run_name': None,
 'top_k': -1,
 'cell_type': 'rnn',
 'n_layer': 4,
 'n_embd': 64,
 'n_embd2': 64,
 'batch_size': 32,
 'learning_rate': 0.001,
 'weight_decay': 0.01}

In [ ]:
# init datasets
train_dataset, test_dataset = create_datasets(args.input_file)
vocab_size = train_dataset.get_vocab_size()
block_size = train_dataset.get_output_length()

print(f"\ndataset determined that: {vocab_size=}, {block_size=}")

number of examples in the dataset: 32033
max word length: 15
number of unique characters in the vocabulary: 26
vocabulary:
abcdefghijklmnopqrstuvwxyz
split up the dataset into 31033 training examples and 1000 test examples

dataset determined that: vocab_size=27, block_size=16


In [ ]:
# init model
model_config = ModelConfig(
    vocab_size=vocab_size,
    block_size=block_size,
    n_layer=args.n_layer,
    n_embd=args.n_embd,
    n_embd2=args.n_embd2
    )

model = RNN(model_config, cell_type=args.cell_type)
model.to(args.device)
print(f"model #params: {sum(p.numel() for p in model.parameters())}")

if args.resume or args.sample_only: # note: if we sample-only then we also assume we are resuming
    print("resuming from existing model in the workdir")
    model.load_state_dict(torch.load(os.path.join(args.work_dir, 'model.pt')))
if args.sample_only:
    print_samples(num=50)
    sys.exit()

model #params: 11803


In [ ]:
# init optimizer
print('Initializing optimizer')
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=args.learning_rate,
                              weight_decay=args.weight_decay,
                              betas=(0.9, 0.99),
                              eps=1e-8)

Initializing optimizer


In [ ]:
# init dataloader
batch_loader = InfiniteDataLoader(train_dataset,
                                  batch_size=args.batch_size,
                                  pin_memory=True,
                                  num_workers=args.num_workers)

# logging
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys if k != 'PAT'} # will be useful for logging
print(config)


if args.wandb_log:
    import wandb
    from google.colab import userdata
    wandb.login(key=userdata.get('WANDB_KEY2'))
    run = wandb.init(project=args.wandb_project,
                     name=args.wandb_run_name,
                     config={**vars(args), **config})

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'vocab_size': 27, 'block_size': 16}


wandb: Currently logged in as: smarginatura (polyphony). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
@torch.inference_mode()
def evaluate(model, dataset, batch_size=50, max_batches=None):
    model.eval()
    loader = DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=0)
    losses = []
    for i, batch in enumerate(loader):
        batch = [t.to(args.device) for t in batch]
        X, Y = batch
        logits = model(X)
        loss = loss_fn(logits, Y)
        losses.append(loss.item())
        if max_batches is not None and i >= max_batches:
            break
    mean_loss = torch.tensor(losses).mean().item()
    model.train() # reset model back to training mode
    return mean_loss

In [ ]:
# training loop

best_loss = None

for step in tqdm(range(args.max_steps)):

    t0 = time.time()

    batch = batch_loader.next()
    batch = [t.to(args.device) for t in batch]
    X, Y = batch
    logits = model(X)
    loss = loss_fn(logits, Y)
    model.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # wait for all CUDA work on the GPU to finish then calculate iteration time taken
    if args.device.startswith('cuda'):
        torch.cuda.synchronize()

    t1 = time.time()

    # logging
    #if step % 10 == 0:
    #    print(f"step {step} | loss {loss.item():.4f} | step time {(t1-t0)*1000:.2f}ms")

    # Evaluate the model
    if step > 0 and step % 500 == 0:
        train_loss = evaluate(model, train_dataset, batch_size=100, max_batches=10)
        test_loss  = evaluate(model, test_dataset,  batch_size=100, max_batches=10)
        if args.wandb_log:
            wandb.log({
                "train/loss": train_loss,
                "test/loss": test_loss,
            }, step=step)
        print(f"step {step} train loss: {train_loss:.4f} test loss: {test_loss:.4f}")

        # Save the model to disk if it has improved
        if best_loss is None or test_loss < best_loss:
            out_path = os.path.join(args.work_dir, "model.pt")
            print(f"test loss {test_loss:4f} is the best so far, saving model to {out_path}")
            torch.save(model.state_dict(), out_path)
            best_loss = test_loss

    # sample from the model
    #if step > 0 and step % 1000 == 0:
    #    print_samples(num=10)

  0%|          | 0/50000 [00:00<?, ?it/s]

step 500 train loss: 2.2501 test loss: 2.2297
test loss 2.229686 is the best so far, saving model to out/model.pt
step 1000 train loss: 2.2134 test loss: 2.1800
test loss 2.180049 is the best so far, saving model to out/model.pt
step 1500 train loss: 2.1632 test loss: 2.1539
test loss 2.153885 is the best so far, saving model to out/model.pt
step 2000 train loss: 2.1263 test loss: 2.1346
test loss 2.134613 is the best so far, saving model to out/model.pt
step 2500 train loss: 2.1561 test loss: 2.1211
test loss 2.121068 is the best so far, saving model to out/model.pt
step 3000 train loss: 2.1335 test loss: 2.1119
test loss 2.111885 is the best so far, saving model to out/model.pt
step 3500 train loss: 2.1088 test loss: 2.1016
test loss 2.101563 is the best so far, saving model to out/model.pt
step 4000 train loss: 2.1096 test loss: 2.1020
step 4500 train loss: 2.1060 test loss: 2.0904
test loss 2.090446 is the best so far, saving model to out/model.pt
step 5000 train loss: 2.0930 test 

* visualize the the training loss curve and test loss curve
(in the notebook, not referring to the wandb board)


In [ ]:
def print_samples(model, num=10):
    """ samples from the model and pretty prints the decoded samples """
    X_init = torch.zeros(num, 1, dtype=torch.long).to(args.device)
    top_k = args.top_k if args.top_k != -1 else None
    steps = train_dataset.get_output_length() - 1 # -1 because we already start with <START> token (index 0)
    X_samp = model.generate(X_init, steps, top_k=top_k, do_sample=True).to('cpu')
    train_samples, test_samples, new_samples = [], [], []
    for i in range(X_samp.size(0)):
        # get the i'th row of sampled integers, as python list
        row = X_samp[i, 1:].tolist() # note: we need to crop out the first <START> token
        # token 0 is the <STOP> token, so we crop the output sequence at that point
        crop_index = row.index(0) if 0 in row else len(row)
        row = row[:crop_index]
        word_samp = train_dataset.decode(row)
        # separately track samples that we have and have not seen before
        if train_dataset.contains(word_samp):
            train_samples.append(word_samp)
        elif test_dataset.contains(word_samp):
            test_samples.append(word_samp)
        else:
            new_samples.append(word_samp)
    print('-'*80)
    for lst, desc in [(train_samples, 'in train'), (test_samples, 'in test'), (new_samples, 'new')]:
        print(f"{len(lst)} samples that are {desc}:")
        for word in lst:
            print(word)
    print('-'*80)


print_samples(model, num=20)

--------------------------------------------------------------------------------
3 samples that are in train:
doni
evan
daniyah
0 samples that are in test:
17 samples that are new:
yaine
kemo
lerade
rohanly
yuchimie
emarica
faton
contar
nicanon
nalizee
malisha
jeelabelligatt
keccel
noun
dravan
rohani
emilissa
--------------------------------------------------------------------------------


In [ ]:
run.finish()

test/loss,█▅▄▄▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▂▁▂▁▁▂▁▁▁▂▁▁▂▁▂▂▂
test/loss,2.03571
train/loss,2.02062


In [3]:
import wandb

In [4]:
api = wandb.Api()
run = api.run('polyphony/name-generation/9hegs882')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
df = run.history()
df

,train/loss,_step,_runtime,test/loss,_timestamp
0,2.250057,500,12.467079,2.229686,1.717545e+09
1,2.213424,1000,18.664754,2.180049,1.717545e+09
2,2.163228,1500,25.725207,2.153885,1.717545e+09
3,2.126289,2000,32.991020,2.134613,1.717545e+09
4,2.156054,2500,39.543967,2.121068,1.717545e+09
...,...,...,...,...,...
94,2.028217,47500,618.261670,2.037997,1.717546e+09
95,2.025776,48000,624.081707,2.036029,1.717546e+09
96,2.010953,48500,630.971210,2.034589,1.717546e+09
97,1.978617,49000,636.747215,2.029901,1.717546e+09


In [16]:
import plotly.graph_objects as go

def _plot_series_plotly(fig, series, series_name, series_index=0):
    palette = ['#1B9E77', '#D95F02', '#7570B3', '#E7298A', '#66A61E', '#E6AB02', '#A6761D', '#666666']
    xs = series['_step']
    train_ys = series['train/loss']
    dev_ys = series['test/loss']

    fig.add_trace(go.Scatter(x=xs, y=train_ys, mode='lines', name=f'{series_name} train', line=dict(color=palette[series_index % len(palette)])))
    fig.add_trace(go.Scatter(x=xs, y=dev_ys, mode='lines', name=f'{series_name} test', line=dict(color=palette[(series_index + 1) % len(palette)])))

# Sort DataFrame
df_sorted = df.sort_values('_step', ascending=True)

# Create Plotly figure
fig = go.Figure()

# Plot the series
_plot_series_plotly(fig, df_sorted, '')

# Update layout
fig.update_layout(
    xaxis_title='step',
    yaxis_title='loss',
    template='plotly_white',
)

# Show the figure
fig.show()

